## Setup Dependencies

In [1]:
!pip install groq
!pip install -U llama-stack

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=1df4cae3f57201a32b840fb801f31a009d1a711503748cc6ad81f6ca8164810f
  Stored in directory: /home/vkeval/.cache/pip/wheels/9e/5b/45/29f72e55d87a29426b04b3cfdf20325c079eb97ab74f59017d
Successfully built fire


In [2]:
!UV_SYSTEM_PYTHON=1 llama stack build --template together --image-type venv

uv is not installed, trying to install it.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 34.6 MB/s eta 0:00:00a 0:00:01
Installing dependencies in system Python environment
Using Python 3.12.7 environment at: /home/vkeval/anaconda3
Audited 1 package in 1.07s
Installing pip dependencies
Using Python 3.12.7 environment at: /home/vkeval/anaconda3
Resolved 119 packages in 1.58s                                       
Prepared 50 packages in 9.35s                                            
Uninstalled 3 packages in 99ms
Installed 50 packages in 1.71s                              
 + aiosqlite==0.21.0
 - anyio==4.2.0 (from file:///croot/anyio_1706220167567/work)
 + anyio==4.8.0
 + autoevals==0.0.121
 + backoff==2.2.1
 + braintrust-core==0.0.58
 + chevron==0.14.0
 + chromadb-client==0.6.3
 + datasets==3.3.2
 + deprecated==1.2.18
 + dnspython==2.7.0
 + eval-type-backport==0.2.2
 + faiss-cpu==1.10.0
 + fastapi==0.115.10
 + googleapis-common-protos==1.68.0
 + grpcio==1.70.0
 + httpx-s

In [2]:
!pip install yt-dlp pytubefix youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 2.9 MB/s eta 0:00:00


## Setup Tools

In [3]:
import yt_dlp
from dataclasses import dataclass
from datetime import datetime
from youtube_transcript_api import YouTubeTranscriptApi

@dataclass
class VideoMetadata:
    title : str
    upload_data : str
    duration_s : int
    url : str

def search_youtube(search_query, num_queries=2):
    ydl_opts = {
        "default_search": f"ytsearch{num_queries}",
        "quiet": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        search_results = ydl.extract_info(search_query, download=False)

    return [VideoMetadata(x['title'], datetime.fromtimestamp(float(x['upload_date'])).strftime('%m/%d/%Y'), x['duration'], x['webpage_url']) for x in search_results['entries']]

def get_transcript(url, fast=True):
    if fast:
        vid_id = pytubefix.YouTube(url).video_id
    else:
        with yt_dlp.YoutubeDL({'quiet':True}) as ydl:
            vid_id = ydl.extract_info(url, download=False)['id']
    return YouTubeTranscriptApi.get_transcript(vid_id)

In [1]:
import os
from groq import Groq

with open('api_key', 'r') as f:
    api_key = f.readline()

client = Groq(
    api_key=api_key,  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of low latency LLMs",
        }
    ],
    model="deepseek-r1-distill-llama-70b",
)
print(chat_completion.choices[0].message.content)

<think>
Okay, so I'm trying to understand why low latency is important for large language models (LLMs). I remember reading that latency refers to the delay before a response is received, so low latency means faster responses. But why is that a big deal for LLMs?

First, I think about where LLMs are used. They're in things like chatbots, virtual assistants, and maybe even in real-time applications. So if someone is using a chatbot and asks a question, they don't want to wait a long time for an answer. If the latency is high, the user experience would be slow and frustrating. That makes sense. People expect quick responses, especially when they're interacting in real-time, like in a conversation.

Then there's real-time applications. I'm not entirely sure what qualifies as a real-time application, but maybe things like live translation or live subtitles. If you're translating speech in real-time, any delay could make the translation useless because the speaker has already moved on. So l